🔹 1. Nguyên lý hoạt động (không công thức nặng!)
🎯 Mục tiêu:
Dự đoán nhãn của một điểm mới dựa trên nhãn của những điểm "gần nó nhất" trong tập huấn luyện.

🧠 Ý tưởng chính — "Hãy xem hàng xóm của bạn là ai!":
"Bạn là trung bình của 5 người bạn thân nhất."
→ KNN áp dụng điều này theo nghĩa hình học. 

🔍 Cách hoạt động:
Khi có một điểm mới (chưa biết nhãn),
Tính khoảng cách từ điểm đó đến tất cả các điểm trong tập huấn luyện (thường dùng khoảng cách Euclid).
Chọn ra K điểm gần nhất (K là số bạn chọn trước, ví dụ K=5).
Đa số thắng: Nhãn nào xuất hiện nhiều nhất trong K điểm đó → đó là dự đoán.
💡 Ví dụ:
Bạn có một bông hoa mới, đo được:
sepal_length=5.0, petal_length=1.5
Trong dữ liệu huấn luyện, 5 bông gần nhất đều là setosa
→ Dự đoán: setosa
✅ Ưu điểm:
Không cần "huấn luyện" theo nghĩa truyền thống → lưu toàn bộ dữ liệu, "học" ngay khi dự đoán.
Không giả định gì về phân phối dữ liệu → hoạt động tốt với mối quan hệ phi tuyến phức tạp.
Dễ hiểu, dễ cài đặt.
❌ Hạn chế:
Rất chậm khi dự đoán nếu dữ liệu lớn (phải tính khoảng cách với toàn bộ tập huấn luyện).
Nhạy cảm với đặc trưng có đơn vị khác nhau (ví dụ: tuổi vs thu nhập) → cần chuẩn hóa.
Hiệu quả giảm khi số chiều (số đặc trưng) rất lớn → "lời nguyền chiều không gian" (curse of dimensionality).

In [1]:
import numpy as np
from collections import Counter

class SimpleKNN:
    def __init__(self, k=3):
        self.k = k

    def fit(self, X, y):
        # KNN "học" bằng cách lưu toàn bộ dữ liệu
        self.X_train = np.array(X)
        self.y_train = np.array(y)

    def _euclidean_distance(self, a, b):
        return np.sqrt(np.sum((a - b) ** 2))

    def predict(self, X):
        X = np.array(X)
        predictions = []
        for x in X:
            # Tính khoảng cách đến mọi điểm huấn luyện
            distances = [self._euclidean_distance(x, x_train) for x_train in self.X_train]
            # Lấy k chỉ số có khoảng cách nhỏ nhất
            k_indices = np.argsort(distances)[:self.k]
            k_nearest_labels = [self.y_train[i] for i in k_indices]
            # Chọn nhãn phổ biến nhất
            most_common = Counter(k_nearest_labels).most_common(1)
            predictions.append(most_common[0][0])
        return np.array(predictions)

# --- Thử nghiệm với dữ liệu chuẩn hóa ---
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# Tải dữ liệu
iris = load_iris()
X, y = iris.data, iris.target

# Chia dữ liệu
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# ⚠️ KNN rất nhạy với thang đo → chuẩn hóa!
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Huấn luyện mô hình tự code
knn_model = SimpleKNN(k=5)
knn_model.fit(X_train_scaled, y_train)
y_pred = knn_model.predict(X_test_scaled)

acc = accuracy_score(y_test, y_pred)
print(f"✅ Độ chính xác (KNN tự code): {acc:.2%}")

# So sánh với sklearn
from sklearn.neighbors import KNeighborsClassifier
sk_knn = KNeighborsClassifier(n_neighbors=5)
sk_knn.fit(X_train_scaled, y_train)
sk_acc = sk_knn.score(X_test_scaled, y_test)
print(f"🔍 Sklearn KNN độ chính xác: {sk_acc:.2%}")

✅ Độ chính xác (KNN tự code): 100.00%
🔍 Sklearn KNN độ chính xác: 100.00%


🔹 3. Khi nào dùng KNN?
Dữ liệu
nhỏ hoặc trung bình
(dưới 10k mẫu)
Dữ liệu
rất lớn
→ dự đoán
quá chậm
Mối quan hệ giữa đặc trưng và nhãn
phi tuyến, phức tạp
Số đặc trưng
rất lớn
(trên 100) → khoảng cách mất ý nghĩa
Bạn cần
mô hình đơn giản, không giả định
Cần
giải thích mô hình
(KNN không cho biết "tại sao")
Làm
baseline nhanh
hoặc
kiểm tra chất lượng dữ liệu
Dữ liệu
chưa chuẩn hóa
hoặc có nhiễu mạnh

